In [1]:
import openai
import json
from datetime import datetime
import os

openai.api_key = os.getenv("OPENAI_API_KEY", "")

In [2]:
model = "gpt-4o"
temperatures = [0.5, 0.9]
policy_topics = ["Infrastructure", "Healthcare Reform", "Immigration Reform", "Climate Change", "Israel-US AI Center Act"]

In [3]:
# Prompts and results
baseline_prompt_template = "Write a 5-paragraph policy summary explaining the importance of {}. Use a formal tone."

structured_npf_prompt_template = """You are a policy analyst trained in the Narrative Policy Framework. Write a 5-paragraph policy narrative advocating for {}. Include:
- A setting (describe the policy context and stakes)
- Characters (a hero promoting the policy, a villain opposing it, and a victim affected by inaction)
- A plot (describe the conflict and how the policy resolves it)
- A moral (what lesson or value the audience should take away)
Use vivid, persuasive language. Label each section clearly."""

speech_npf_prompt_template = """You are a skilled policy speechwriter trained in the Narrative Policy Framework. Write a persuasive 5-paragraph policy speech advocating for {}.
The speech should:
- Open with a vivid policy setting that conveys the stakes.
- Introduce key characters: a person or group championing the policy (hero), those resisting it (villain), and those affected by inaction (victims).
- Use a compelling plot to describe the conflict and how the policy addresses it.
- Close with a strong moral message—a lesson or value the audience should remember.
Do not label these parts. Instead, weave them naturally into a rhetorically powerful, emotionally resonant speech in a formal tone, suitable for delivery on the House or Senate floor."""

results = []

In [4]:
# Loop through temperatures, topics, and prompt types
for temp in temperatures:
    for topic in policy_topics:
        for prompt_type, template in [
            ("Baseline", baseline_prompt_template),
            ("Structured NPF", structured_npf_prompt_template),
            ("Speech-style NPF", speech_npf_prompt_template)
        ]:
            prompt = template.format(topic)
            try:
                response = openai.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=temp
                )
                content = response.choices[0].message.content
            except Exception as e:
                content = f"ERROR: {str(e)}"

            results.append({
                "timestamp": datetime.utcnow().isoformat(),
                "topic": topic,
                "prompt_type": prompt_type,
                "prompt": prompt,
                "model": model,
                "temperature": temp,
                "response": content
            })

In [5]:
# Save to JSON
with open("policy_narratives_output.json", "w", encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("Responses saved to 'policy_narratives_output.json'")

Responses saved to 'policy_narratives_output.json'


In [7]:
# Generate Markdown table and save
markdown_lines = [
    "| Topic | Prompt Type | Temperature | Response |",
    "|-------|-------------|-------------|----------|"
]

for r in results:
    topic = r["topic"]
    prompt_type = r["prompt_type"]
    temperature = r["temperature"]
    response = r["response"].replace("\n", " ").replace("|", "\\|")
    markdown_lines.append(f"| {topic} | {prompt_type} | {temperature} | {response} |")

markdown_table = "\n".join(markdown_lines)

with open("outputs_cleaned.md", "w") as f:
    f.write(markdown_table)